In [19]:
import os
import json
import argparse
import time

import numpy as np

import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn

import matplotlib
import matplotlib.pyplot as plt

from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import DataLoader
from tqdm import tqdm


# Remove warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from scipy.sparse import SparseEfficiencyWarning
warnings.simplefilter('ignore', SparseEfficiencyWarning)

from utils.graph_utils import *
from utils.google_tsp_reader import GoogleTSPReader
from architecture.gcn_model import ResidualGatedGCNModel
from utils.model_utils import *

%reload_ext autoreload
%autoreload 2

In [2]:
if torch.cuda.is_available():
    print("CUDA available")
    DEVICE = "cuda"
    dtypeFloat = torch.cuda.FloatTensor
    dtypeLong = torch.cuda.LongTensor
    torch.cuda.manual_seed(1)

CUDA available


# Configs

In [13]:
BATCH_SIZE = 1
LR = 0.001
BEAM_SIZE= 50
DECAY_RATE = 1.01
VAL_LOSS_OLD= 1e6

# Load Data

In [5]:
instances = np.load('/home/aymenkallala/TSP_DL/data/instances.npy')
instances_orders = np.load('/home/aymenkallala/TSP_DL/data/instances_orders_unpadded.npy')

X,y = instances,instances_orders

In [6]:
from sklearn.model_selection import train_test_split
from scipy.spatial import distance_matrix


def split_train_test(X,y,split):
    """Split the dataset into trainset and testset, with each instances linked to its groundtruth solution"""

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split, random_state=42)
    train_dataset = list(zip(X_train,y_train))
    test_dataset = list(zip(X_test,y_test))
    return train_dataset,test_dataset

def collate_batch_gcn(batch):

    """ The necessary modifications to feed the data in the right format for the Graph Conv Net."""
    x_edges = []
    x_edges_values = []
    x_nodes = []
    x_nodes_coord = []  
    y_edges = []
    y_nodes = []

    for x,y in batch:
        W = np.ones((len(y),len(y)),dtype=np.int64)
        np.fill_diagonal(W, 2)

        x_nodes_coord.append(x)
        x_nodes.append(np.ones(len(y),dtype=np.int64))
        y_nodes.append(y)
        x_edges.append(W) #Graph is fully connected
        x_edges_values.append(distance_matrix(x,x))

        #Compute the adjacency matrix
        edges_target = np.zeros((len(y),len(y)),dtype=np.int64)
        for idx in range(len(y) - 1):
                i = y[idx]
                j = y[idx + 1]
                edges_target[i][j] = 1
        edges_target[y[-1]][y[0]] = 1

        y_edges.append(edges_target)
    
    return (torch.LongTensor(x_edges).to(DEVICE),
            torch.FloatTensor(x_edges_values).to(DEVICE),
            torch.LongTensor(x_nodes).to(DEVICE),
            torch.FloatTensor(x_nodes_coord).to(DEVICE),
            torch.LongTensor(y_edges).to(DEVICE),
            torch.LongTensor(y_nodes).to(DEVICE))

In [17]:
train_dataset,test_dataset = split_train_test(X,y,0.1)

train_dataloader = DataLoader(train_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        collate_fn=collate_batch_gcn)

test_dataloader = DataLoader(test_dataset,
                        batch_size=5,
                        shuffle=True,
                        collate_fn=collate_batch_gcn)

# Test forward pass

In [8]:
net = nn.DataParallel(ResidualGatedGCNModel(dtypeFloat, dtypeLong))
if torch.cuda.is_available():
        net.cuda()
print(net)

nb_param = 0
for param in net.parameters():
        nb_param += np.prod(list(param.data.size()))
print('Number of parameters:', nb_param)

DataParallel(
  (module): ResidualGatedGCNModel(
    (nodes_coord_embedding): Linear(in_features=2, out_features=300, bias=False)
    (edges_values_embedding): Linear(in_features=1, out_features=150, bias=False)
    (edges_embedding): Embedding(3, 150)
    (gcn_layers): ModuleList(
      (0): ResidualGatedGCNLayer(
        (node_feat): NodeFeatures(
          (U): Linear(in_features=300, out_features=300, bias=True)
          (V): Linear(in_features=300, out_features=300, bias=True)
        )
        (edge_feat): EdgeFeatures(
          (U): Linear(in_features=300, out_features=300, bias=True)
          (V): Linear(in_features=300, out_features=300, bias=True)
        )
        (bn_node): BatchNormNode(
          (batch_norm): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        )
        (bn_edge): BatchNormEdge(
          (batch_norm): BatchNorm2d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        )
      )
      (1): Re

In [14]:
for i,(x_edges,x_edges_values,x_nodes,x_nodes_coord,y_edges,y_nodes ) in enumerate(test_dataloader):
    # Compute class weights
    edge_labels = y_edges.cpu().numpy().flatten()
    edge_cw = compute_class_weight("balanced", classes=np.unique(edge_labels), y=edge_labels)
    #print("Class weights: {}".format(edge_cw))


    y_preds, loss = net.forward(x_edges, x_edges_values, x_nodes, x_nodes_coord, y_edges, edge_cw)
    loss = loss.mean()
    #print("Output size: {}".format(y_preds.size()))
    #print("Loss value:", loss)

    bs_nodes = beamsearch_tour_nodes_shortest(
            y_preds, x_edges_values, BEAM_SIZE, BATCH_SIZE, 50, dtypeFloat, dtypeLong, probs_type='logits')
    pred_tour_len = mean_tour_len_nodes(x_edges_values, bs_nodes)
    gt_tour_len = mean_tour_len_nodes(x_edges_values,y_nodes)

    print(bs_nodes)
    print(pred_tour_len)
    print(gt_tour_len)
        

    if i ==5:
        break
    

tensor([[ 0, 31, 40, 35, 33, 14, 32, 19, 15,  3, 17,  1,  4,  2, 44, 43, 29, 36,
         23, 27, 21, 39, 11, 38,  9,  6, 41, 25, 28,  8, 16, 48, 22, 34, 42, 47,
         26, 49,  7, 24, 18, 20, 13,  5, 46, 12, 37, 30, 45, 10]],
       device='cuda:0')
38.77392625808716
10.12177898734808
tensor([[ 0, 43, 19, 47, 14, 39, 27, 40,  6, 41, 29, 48,  3, 45, 33, 49, 30, 44,
          9, 46, 13, 42, 36, 25, 37, 12, 21,  4, 17, 31, 26,  1,  8,  7, 34, 22,
          2, 20, 28, 11,  5, 32, 23, 18, 15, 10, 38, 35, 16, 24]],
       device='cuda:0')
61.351875737309456
9.168619956821203
tensor([[ 0, 44, 19, 48, 34, 46, 11, 43, 10, 40,  1, 45, 13, 49, 32, 41,  7, 47,
         18, 42, 15, 21, 36,  5, 20, 27, 12, 14, 22, 17, 35, 23, 31, 24, 28,  4,
         25,  3, 29,  2, 33, 39, 37, 16, 38, 26,  6,  8, 30,  9]],
       device='cuda:0')
57.609626583755016
9.941556937992573
tensor([[ 0, 31,  1, 47,  6, 43,  7, 32, 20, 45,  4, 46, 11, 28, 14, 22, 12, 39,
         13, 29, 10, 35, 15, 41, 18, 36, 17, 42,  

# Training loop

In [30]:
def train_gcn(dataloader,net, optimizer,epoch):
    # Set training mode
    net.train()
    # Initially set loss class weights as None
    edge_cw = None
    log_interval = 100

    # Initialize running data
    running_loss = 0.0
    running_nb_data = 0.0

    for idx,batch in tqdm(enumerate(dataloader)):

        x_edges,x_edges_values,x_nodes,x_nodes_coord,y_edges,y_nodes=batch
        
        # Compute class weights (if uncomputed)
        if type(edge_cw) != torch.Tensor:
            edge_labels = y_edges.cpu().numpy().flatten()
            edge_cw = compute_class_weight("balanced", classes=np.unique(edge_labels), y=edge_labels)
        
        # Forward pass
        y_preds, loss = net.forward(x_edges, x_edges_values, x_nodes, x_nodes_coord, y_edges, edge_cw)
        loss = loss.mean()
        loss.backward()

        # Backward pass
        optimizer.step()
        optimizer.zero_grad()


        # Update running data
        running_nb_data += len(batch)
        running_loss += len(batch)* loss.item() # Re-scale loss
        
        
        if idx % log_interval == 0 and idx > 0:
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches".format(epoch, idx, len(dataloader)),
                f"| loss {running_loss/running_nb_data}"
            )
            running_loss, running_nb_data = 0, 0


def test(dataloader,net,epoch):

    net.eval()
    edge_cw = None
    log_interval = 10

    # Initialize running data
    running_loss = 0.0
    running_nb_data = 0.0
    running_tour_length = 0.0
    running_gt_length = 0.0

    #Predictions

    with torch.no_grad():
        for idx,batch in tqdm(enumerate(dataloader)):
            x_edges,x_edges_values,x_nodes,x_nodes_coord,y_edges,y_nodes=batch

            if type(edge_cw) != torch.Tensor:
                edge_labels = y_edges.cpu().numpy().flatten()
                edge_cw = compute_class_weight("balanced", classes=np.unique(edge_labels), y=edge_labels)
            
            # Forward pass
            y_preds, loss = net.forward(x_edges, x_edges_values, x_nodes, x_nodes_coord, y_edges, edge_cw)
            loss = loss.mean()

            #Beamsearch decoding
            bs_nodes = beamsearch_tour_nodes_shortest(
            y_preds, x_edges_values, BEAM_SIZE, BATCH_SIZE, 50, dtypeFloat, dtypeLong, probs_type='logits')
            pred_tour_len = total_tour_len_nodes(x_edges_values, bs_nodes)
            gt_tour_len = total_tour_len_nodes(x_edges_values,y_nodes)


            # Update running data
            running_nb_data += len(batch)
            running_loss += len(batch)* loss.item() # Re-scale loss
            running_tour_length +=pred_tour_len
            running_gt_length += gt_tour_len

        
        
            if idx % log_interval == 0 and idx > 0:
                print(
                    "| epoch {:3d} | {:5d}/{:5d} batches".format(epoch, idx, len(dataloader)),
                    f"| loss {running_loss/running_nb_data}"
                    f"| gap {100*(running_tour_length-running_gt_length)/running_gt_length}"
                )
                running_loss, running_nb_data,running_tour_length,running_gt_length = 0,0,0,0






In [31]:
test(test_dataloader,net,1)

0it [00:00, ?it/s]

12it [00:02,  4.98it/s]

| epoch   1 |    10/ 2170 batches | loss 0.8697825236753984| pred tour length 718.0376259796321| gt tour length 529.10810290277| gap 35.70716873175162


22it [00:04,  4.98it/s]

| epoch   1 |    20/ 2170 batches | loss 0.8726309239864349| pred tour length 790.6099488437176| gt tour length 467.8603347092867| gap 68.98417971999638


...

| epoch   1 |   100/ 2170 batches | loss 0.8780515134334564| pred tour length 796.7968850322068| gt tour length 466.63018373027444| gap 70.75553892861292


111it [00:22,  4.93it/s]

| epoch   1 |   110/ 2170 batches | loss 0.8634570956230163| pred tour length 630.8461493961513| gt tour length 473.2883401811123| gap 33.2900255169495


119it [00:24,  4.95it/s]


KeyboardInterrupt: 